In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
plt.style.use('matplotlibrc')

In [33]:
cases = pd.read_csv('cases.csv')['cases'].values
covars = pd.read_csv('covars.csv')
covars = covars[['seas1', 'seas2', 'seas3', 'seas4', 'seas5', 'seas6']].values[:cases.shape[0],:]

 rho           tau         beta1         beta2         beta3         beta4         beta5         beta6            nu         gamma 
 7.617542e-01  2.660769e+02  1.437621e+00  1.290241e+00  1.167740e+00  1.275811e+00  1.361995e+00  1.153047e+00  9.648961e-01  3.500000e+00 
        sigma        theta0         alpha            mu         delta        sig_sq           S_0           E_0           I_0           A_0 
 5.000000e+00  0.000000e+00  2.397300e-03  4.287000e-04  1.433000e-04  1.041514e-01  9.993196e-01  1.834793e-04  4.969569e-04  0.000000e+00 
          R_0         pop_0         betat 
 0.000000e+00  1.091182e+07 -6.717172e-02 
 
 
 beta1 = .02, beta2 = .02, beta3 = .02,
                beta4 = .02, beta5 = .02, beta6 = .02,
                tau = 0.02, rho = 0.02, nu = 0.02, sig_sq = 0.02,
                E_0 = ivp(0.2), I_0 = ivp(0.2)

In [77]:
params = pd.read_csv('params.csv')
params.T

,0
rho,7.617542e-01
tau,2.660769e+02
beta1,1.437621e+00
beta2,1.290241e+00
beta3,1.167740e+00
beta4,1.275811e+00
beta5,1.361995e+00
beta6,1.153047e+00
nu,9.648961e-01
gamma,3.500000e+00


In [ ]:
def pfilter(mod, Np=50):
    

In [58]:
torch.cat([torch.tensor(0.0).reshape(1), torch.ones(3)])

tensor([0., 1., 1., 1.])

In [83]:
class Haiti1(torch.jit.ScriptModule):
    def __init__(self, covars, cases, params, dt):
        super().__init__()
        self.cases = torch.tensor(cases)
        self.covars = torch.tensor(covars)
        
        #trainable params
        self.betas = nn.Parameter(torch.tensor([params['beta1'], params['beta2'], 
                                                params['beta3'], params['beta4'], 
                                                params['beta5'], params['beta6']]).squeeze())
        self.betat = nn.Parameter(torch.tensor(params['betat']))
        self.rho = nn.Parameter(torch.tensor(params['rho']))
        self.tau = nn.Parameter(torch.tensor(params['tau']))
        self.nu = nn.Parameter(torch.tensor(params['nu']))
        self.sig_sq = nn.Parameter(torch.tensor(params['sig_sq']))
        self.E_0 = nn.Parameter(torch.tensor(params['E_0']))
        self.I_0 = nn.Parameter(torch.tensor(params['I_0']))
        
        #rinit
        self.S = torch.tensor(params['S_0'])
        self.E = self.E_0
        self.I = self.I_0
        self.R = torch.tensor(params['R_0'])
        self.incid = 0
        self.mu = torch.tensor(params['mu'])
        self.delta = torch.tensor(params['delta'])
        self.sigma = torch.tensor(params['sigma'])
        self.t = 0
        self.dt = dt
        
    #@torch.jit.script_method
    def reulermultinom(self, N, rates, dt):
        sumrates = torch.sum(rates)
        p0 = torch.exp(-sumrates * dt)
        return torch.distributions.Multinomial(torch.cat([p0.reshape(1), (1-p0)*rates/sumrates]), N).sample()
        
    
    #@torch.jit.script_method
    def step(self):
        pop = self.S + self.E + self.I + self.R
        births = torch.distributions.Poisson(self.mu).sample() * pop
        beta = torch.exp(torch.dot(self.betas, self.covars[self.t]) + (self.betat-215)/(430-215))
        foi = torch.pow(self.I, self.nu) * beta / pop * torch.distributions.Gamma(self.sig_sq, self.dt).sample() / self.dt
        
        s_rates = torch.cat([foi, self.delta])
        e_rates = torch.cat([self.sigma, self.delta])
        print(s_rates)
        
        
        #deaths = torch.distributions.Poisson(self.delta).sample()
        self.t += 1
        

In [84]:
mod1 = Haiti1(covars, cases, params, 1/7)
mod1.step()

tensor([7.9815e-15, 1.4330e-04], dtype=torch.float64, grad_fn=<CatBackward0>)


In [ ]:
# compute rates
  s_rates <- c("Srate[0] = foi; \n ", "Srate[1] = delta; \n ")
  e_rates <- c("Erate[0] = sigma*(1-theta0); \n ", "Erate[1] = sigma*theta0; \n ", "Erate[2] = delta; \n ")
  i_rates <- c("Irate[0] = gamma; \n ", "Irate[1] = delta; \n ")
  a_rates <- c("Arate[0] = gamma; \n ", "Arate[1] = delta; \n ")
  r_rates <- c("Rrate[0] = alpha; \n ", "Rrate[1] = delta; \n ")
  if (depts > 1) {
    s_rates <- c(s_rates, paste0("Srate[", 2:(depts + 1), "] = eta", 1:depts, "; \n "),
                 paste0("S", 1:depts, "rate[0] = foi; \n ",
                        "S", 1:depts, "rate[1] = delta; \n "))
    e_rates <- c(e_rates, paste0("Erate[", 3:(depts + 2), "] = eta", 1:depts, "; \n "),
                 paste0("E", 1:depts, "rate[0] = sigma*(1-theta", 1:depts, "); \n ",
                        "E", 1:depts, "rate[1] = sigma*theta", 1:depts, "; \n ",
                        "E", 1:depts, "rate[2] = delta; \n "))
    i_rates <- c(i_rates, paste0("Irate[", 2:(depts + 1), "] = eta", 1:depts, "; \n "),
                 paste0("I", 1:depts, "rate[0] = gamma; \n ",
                        "I", 1:depts, "rate[1] = delta; \n "))
    a_rates <- c(a_rates, paste0("Arate[", 2:(depts + 1), "] = eta", 1:depts, "; \n "),
                 paste0("A", 1:depts, "rate[0] = gamma; \n ",
                        "A", 1:depts, "rate[1] = delta; \n "))
    r_rates <- c(r_rates, paste0("Rrate[", 2:(depts + 1), "] = eta", 1:depts, "; \n "),
                 paste0("R", 1:depts, "rate[0] = alpha; \n ",
                        "R", 1:depts, "rate[1] = delta; \n "))
  }